## Example 1

In [ ]:
library(httr)

url = "https://omabrowser.org/api/protein/P53_RAT/"
response = GET(url)

response_content_list = httr::content(response, as = "parsed")


## Example 2


In [ ]:
library(OmaDB)

Sequence <- 'MKLVFLVLLFLGALGLCLAGRRRSVQWCAVSQPEATKCFQWQRNMRKVRGPPVSCIKRDSPIQCIQAIAENRADAVTLDGGFIYEAGLAPYKLRPVAAEVYGTERQPRTHYYAVAVVKKGGSFQLNELQGLKSCHTGLRRTAGWNVPIGTLRPFLNWTGPPEPIEAAVARFFSASCVPGADKGQFPNLCRLCAGTGENKCAFSSQEPYFSYSGAFKCLRDGAGDVAFIRESTVFEDLSDEAERDEYELLCPDNTRKPVDKFKDCHLARVPSHAVVARSVNGKEDAIWNLLRQAQEKFGKDKSPKFQLFGSPSGQKDLLFKDSAIGFSRVPPRIDSGLYLGSGYFTAIQNLRKSEEEVAARRARVVWCAVGEQELRKCNQWSGLSEGSVTCSSASTTEDCIALVLKGEADAMSLDGGYVYTAGKCGLVPVLAENYKSQQSSDPDPNCVDRPVEGYLAVAVVRRSDTSLTWNSVKGKKSCHTAVDRTAGWNIPMGLLFNQTGSCKFDEYFSQSCAPGSDPRSNLCALCIGDEQGENKCVPNSNERYYGYTGAFRCLAENAGDVAFVKDVTVLQNTDGNNNEAWAKDLKLADFALLCLDGKRKPVTEARSCHLAMAPNHAVVSRMDKVERLKQVLLHQQAKFGRNGSDCPDKFCLFQSETKNLLFNDNTECLARLHGKTTYEKYLGPQYVAGITNLKKCSTSPLLEACEFLRK'
seq_annotation = mapSequence(sequence)

seq_annotation$targets[[1]]$canonicalid          # 'YEAST00058'

hog_id <- seq_annotation$targets[[1]]$oma_hog_id   # ’HOG:0413862.1a.1b'
hog <- getHOG(id = hog_id, members = TRUE, level = 'Mammalia')
hog$members


hog <- getHOG(id = 'TRFL_HUMAN', members = TRUE, level = 'Mammalia')

ontologies <- getInfo(hog$members, type = 'ontologies')

bgHOG <- getHOG(id = 'TRFL_HUMAN', members = TRUE, level = 'Eukaryota')
bgOntologies <- getInfo(bgHOG$members, type = 'ontologies')

library(topGO)
myGO <- getTopGO(annotations = bgOntologies, format = 'geneID2GO',  
    myInterestingGenes = hog$members, ontology = 'BP')
myRes <- runTest(myGO, algorithm = 'classic', statistic = 'fisher')
print(GenTable(myGO, myRes))


## Example 3

In [ ]:
library(OmaDB)

tax = getTaxonomy(root = 'Hominoidea')

library(ggtree)

tree = getTree(tax$newick)

mytree = ggtree(tree) 

labels = c("Pan troglodytes",
              "Pan paniscus",
              "Homo sapiens",
              "Gorilla gorilla gorilla",
              "Pongo abelii",
              "Nomascus leucogenys")

library(rphylopic)

ids = sapply(labels, FUN = function(x)         name_search(x)$canonicalName[1,1])

images = sapply(as.character(ids), FUN = function(x) tryCatch(name_images(x)$same[[1]]$uid, error = function(w) name_images(x)$supertaxa[[1]]$uid))


d <- data.frame(label = labels, images = as.character(images))

library(dplyr)

mytree %<+% d + geom_tiplab(aes(image=images), geom="phylopic", offset = 2.3, color='steelblue') + geom_tiplab(offset = 0.3) + ggplot2::xlim(0, 7)



## Example 4

In [ ]:
library(OmaDB)

mouse = getTaxonomy(members='Mus musculus',newick=FALSE)$id
human = getTaxonomy(members='Homo sapiens',newick=FALSE)$id
dog = getTaxonomy(members='Canis lupus familiaris',newick=FALSE)$id

human_mouse = getPairs(genome_id1 =human, genome_id2 = mouse, per_page = "all", rel_type="1:1")

human_dog = getPairs(genome_id1 =human, genome_id2 = dog, per_page = "all", rel_type="1:1")

human_mouse$Species  = 'Mus musculus'
human_dog$Species = 'Canis lupus familiaris'

all_pairs = rbind(human_mouse,human_dog)
all_pairs$Species = as.factor(all_pairs$Species)
library(ggplot2)

g <- ggplot(all_pairs, aes(x = 'evolutionary distance [PAM]', fill = Species)) + geom_density(alpha = 0.5) + theme(legend.position="bottom",panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
panel.background = element_blank(), axis.line = element_line(colour = "black"))
dev.off()

print(g)

ks.test(human_dog$distance,human_mouse$distance) 


## Example 5

In [ ]:
# Our mystery sequence is cystic fibrosis transmembrane conductance
# regulator in the Emperor penguin (UniProt ID: A0A087RGQ1_APTFO)
mysterySeq <- 'FFFLLRWTKPILRKGYRRRLELSDIYQIPSADSADNLSEKLEREWDRELATSKKKPKLINALRRCFFWKFMFYGIILYLGEVTKSVQPLLLGRIIASYDPDNSDERSIAYYLAIGLCLLFLVRTLLIHPAIFGLHHIGMQMRIAMFSLIYKKILKLSSRVLDKISTGQLVSLLSNNLNKFDEGLALAHFVWIAPLQVALLMGLLWDMLEASAFSGLAFLIVLAFFQAWLGQRMMKYRNKRAGKINERLVITSEIIENIQSVKAYCWEDAMEKMIESIRETELKLTRKAAYVRYFNSSAFFFSGFFVVFLAVLPYAVIKGIILRKIFTTISFCIVLRMTVTRQFPGSVQTWYDSIGAINKIQDFLLKKEYKSLEYNLTTTGVELDKVTAFWDEGIGELFVKANQENNNSKAPSTDNNLFFSNFPLHASPVLQDINFKIEKGQLLAVSGSTGAGKTSLLMLIMGELEPSQGRLKHSGRISFSPQVSWIMPGTIKENIIFGVSYDEYRYKSVIKACQLEEDISKFPDKDYTVLGDGGIILSGGQRARISLARAVYKDADLYLLDSPFGHLDIFTEKEIFESCVCKLMANKTRILVTSKLEHLKIADKILILHEGSCYFYGTFSELQGQRPDFSSELMGFDSFDQFSAERRNSILTETLRRFSIEGEGTGSRNEIKKQSFKQTSDFNDKRKNSIIINPLNASRKFSVVQRNGMQVNGIEDGHNDPPERRFSLVPDLEQGDVGLLRSSMLNTDHILQGRRRQSVLNLMTGTSVNYGPNFSKKGSTTFRKMSMVPQTNLSSEIDIYTRRLSRDSVLDITDEINEEDLKECFTDDAESMGTVTTWNTYFRYVTIHKNLIFVLILCVTVFLVEVAASLAGLWFLKQTALKANTTQSENSTSDKPPVIVTVTSSYYIIYIYVGVADTLLAMGIFRGLPLVHTLITVSKTLHQKMVHAVLHAPMSTFNSWKAGGMLNRFSKDTAVLDDLLPLTVFDFIQLILIVIGAITVVSILQPYIFLASVPVIAAFILLRAYFLHTSQQLKQLESEARSPIFTHLVTSLKGLWTLRAFGRQPYFETLFHKALNLHTANWFLYLSTLRWFQMRIEMIFVVFFVAVAFISIVTTGDGSGKVGIILTLAMNIMGTLQWAVNSSIDVDSLMRSVGRIFKFIDMPTEEMKNIKPHKNNQFSDALVIENRHAKEEKNWPSGGQMTVKDLTAKYSEGGAAVLENISFSISSGQRVGLLGRTGSGKSTLLFAFLRLLNTEGDIQIDGVSWSTVSVQQWRKAFGVIPQKVFIFSGTFRMNLDPYGQWNDEEIWKVAEEVGLKSVIEQFPGQLDFVLVDGGCVLSHGHKQLMCLARSVLSKAKILLLDEPSAHLDPVTSQVIRKTLKHAFANCTVILSEHRLEAMLECQRFLVIEDNKLRQYESIQKLLNEKSSFRQAISHADRLKLLPVHHRNSSKRKPRPKITALQEETEEEVQETRL'
myAnnotations <- getAnnotation(mysterySeq)
